<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MC_Tiempo_AyC_vs_CRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
from scipy.integrate import quad

Monte Carlo: Acierto y Error

In [66]:
%%time

def g(x):
  return np.sqrt(np.arctan(x))

aciertos=0
N=10000

for i in range (N):
  x = rd.random()
  y = rd.random()

  if y <= g(x):
    aciertos+=1
  else:
    aciertos+=0

I=aciertos/N
var=(aciertos/N)-(I**2)
print("El valor estimado es: ",I, "\nVarianza: ",var)

El valor estimado es:  0.6306 
Varianza:  0.23294364
CPU times: user 28.7 ms, sys: 7 µs, total: 28.7 ms
Wall time: 28.6 ms


Monte Carlo: Crudo

In [67]:
%%time

def g(x):
  return np.sqrt(np.arctan(x))

def crudo(N):
  G=[]
  for i in range (N):
    u=rd.random()
    G.append(g(u))
  return np.mean(G), np.var(G), np.std(G), G

e=crudo(10000)

print("El valor estimado es: ", e[0], "\nVarianza: ", e[1])

El valor estimado es:  0.628821954969806 
Varianza:  0.042286407327671666
CPU times: user 24.7 ms, sys: 7 µs, total: 24.7 ms
Wall time: 24.3 ms
